## <font color='blue'>AutoML em Python</font>


# <font color='blue'>Machine Learning Automation</font>
# <font color='blue'>AutoML com Python - AutoKeras</font>


## AutoML - Parte 1

## Fonte de Dados

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

## Instalando e Carregando Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.12.4


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q watermark

In [5]:
# Instala o autokeras
!pip install autokeras


^C


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     -------- ------------------------------- 10.2/48.7 kB ? eta -:--:--
     ----------------------- -------------- 30.7/48.7 kB 640.0 kB/s eta 0:00:01
     ------------------------------- ------ 41.0/48.7 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 244.9 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of keras-nlp to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of keras-nlp to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/162.4 kB ? eta -:--:--
   -

In [6]:
# Filtro de warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [7]:
# Imports
import tensorflow as tf
import autokeras as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
%matplotlib inline

ImportError: cannot import name 'pywrap_tensorflow' from 'tensorflow.python' (c:\Users\55839\anaconda3\Lib\site-packages\tensorflow\python\__init__.py)

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Ricardo Roberto de Lima" --iversions

## Carga e Exploração dos Dados

In [ ]:
# Carrega os dados
dataset = fetch_california_housing()

In [ ]:
# Elementos dos dados
dataset.keys()

In [ ]:
# Converte em dataframe
dados = pd.DataFrame(dataset.data, columns = dataset.feature_names)

In [ ]:
# Extrai a variável target
target = pd.Series(dataset.target, name = 'MedPrice')

In [ ]:
# Shape
dados.shape

In [ ]:
# Tipos de dados
dados.dtypes

In [ ]:
# Visualiza
dados.head()

In [ ]:
# Resumo estatístico
pd.options.display.float_format = '{:,.2f}'.format
dados.describe()

In [ ]:
# Verificando valores nulos
dados.isnull().any()

In [ ]:
# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(dados, target, test_size = 0.2, random_state = 42)

In [ ]:
# Shape
print("Shape Atributos em Treino: {}".format(X_treino.shape))
print("Shape Target em Treino: {}".format(y_treino.shape))
print("Shape Atributos em Teste: {}".format(X_teste.shape))
print("Shape Target em Teste: {}".format(y_teste.shape))

## Modelagem

https://autokeras.com/structured_data_regressor/

In [ ]:
# Cria o modelo
# Vamos tentar 10 modelos diferentes
regressor = ak.StructuredDataRegressor(max_trials = 10, overwrite = True)

Agora buscamos o modelo.

In [ ]:
%%time
regressor.fit(x = X_treino, y = y_treino, batch_size = 1024)

## Avaliação do Modelo

In [ ]:
# Previsões
predicted_y = regressor.predict(X_teste)
print(predicted_y)

In [ ]:
# Calcula as métricas
test_loss, test_mse = regressor.evaluate(X_teste, y_teste, verbose = 0)
print('MSE em Teste: ', test_mse)

In [ ]:
!pip install -q pydot

In [ ]:
# Visualiza o melhor modelo
best_model = regressor.export_model()
tf.keras.utils.plot_model(best_model, show_shapes = True, expand_nested = True) 

## Customizando o Aprendizado do Modelo com EarlyStopping

In [ ]:
# Cria o modelo
regressor = ak.StructuredDataRegressor(max_trials = 100)

In [ ]:
# Cria o callback EarlyStopping para interromper o aprendizado quando encontrar o melhor modelo
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10) 

Agora buscamos o melhor modelo.

In [ ]:
%%time
regressor.fit(x = X_treino, 
              y = y_treino, 
              epochs = 200,
              batch_size = 4096, 
              verbose = 2, 
              callbacks = [callback])

In [ ]:
# Calcula as métricas
test_loss, test_mse = regressor.evaluate(X_teste, y_teste, verbose = 0)
print('MSE em Teste: ', test_mse)

In [ ]:
# Visualiza o melhor modelo
best_model = regressor.export_model()
tf.keras.utils.plot_model(best_model, show_shapes = True, expand_nested = True) 

# Terminando o processo 